This is the analysis on brain connectivity data of 5 pairs of symmetrical nodes on certain freq. bands:
1) P3,P4,O1,O2 nodes for delta band 
2) C3,C4,F3,F4 nodes for theta band 
3) C3,C4,F3,F4 nodes for delta band 
4) C3,C4,T3,T4 nodes for theta band
5) C3,C4,P3,P4 nodes for theta band

In [31]:
import mne
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from Classification_models_LOSO_2Class import *
from model_functions import *

In [7]:
def remove_test(features,test):
    features_train = [features[i] for i in range(len(features)) if i not in test]
    #target_train = [targets[i] for i in range(len(targets)) if i not in test]
    return features_train
def remove_test_targets(targets, test):
    target_train = [targets[i] for i in range(len(targets)) if i not in test]
    return target_train

In [8]:
conn_data_1 = np.load('processed_data/con_data_01.npy',allow_pickle=True)
conn_data_2 = np.load('processed_data/con_data_02.npy',allow_pickle=True)
conn_data_3 = np.load('processed_data/con_data_03.npy',allow_pickle=True)
conn_data_4 = np.load('processed_data/con_data_04.npy',allow_pickle=True)
conn_data_5 = np.load('processed_data/con_data_05.npy',allow_pickle=True)

In [32]:
targetss=[]
targetss=[1 for i in range(36)]
targetss.extend([0 for i in range(29)])
test= [4,6,8,20,33,49,53,63]

In [10]:
data_1 = remove_test(conn_data_1, test)
data_2 = remove_test(conn_data_2,test)
data_3 = remove_test(conn_data_3, test)
data_4 = remove_test(conn_data_4,test)
data_5 = remove_test(conn_data_5, test)

In [11]:
targets= remove_test_targets(targetss, test)

In [12]:
targets= np.array(targets)
type(targets)

numpy.ndarray

In [13]:
data_1 = [data_1[i] for i in range(57)]
data_2 = [data_2[i] for i in range(57)]
data_3 = [data_3[i] for i in range(57)]
data_4 = [data_4[i] for i in range(57)]
data_5 = [data_5[i] for i in range(57)]

In [14]:
all_data = [data_1,data_2,data_3,data_4,data_5]

In [15]:
def get_arr(data, idx , epoch): 
    trials = [(data[idx][epoch].reshape(-1,))[i] for i in range(16) if (data[idx][epoch].reshape(-1,))[i] != 0]
    m = len(trials)
    return (np.array(trials)).reshape((1,m))


def get_all_freq_arr(all_data,idx, epoch):
    r = get_arr(all_data[0], idx , epoch)
    for i in range(1,len(all_data)):
        r = np.concatenate((r,get_arr(all_data[i], idx , epoch)),axis =1)
        
    #r=np.concatenate((get_arr(all_data[0], idx , epoch),get_arr(all_data[1], idx , epoch),get_arr(all_data[2], idx , epoch),get_arr(all_data[3], idx , epoch),get_arr(all_data[4], idx , epoch)), axis=1)
    return r
    

In [16]:
def get_all_epochs_data(all_data,idx):
    r = get_all_freq_arr(all_data,idx,0)
    for j in range(1,len(all_data[0][idx])):
        r = np.concatenate((r,get_all_freq_arr(all_data,idx,j)),axis = 0)
    return r

In [17]:
def get_all_features(all_data):
    output_array = [];
    for j in range(len(all_data[0])):
        r = get_all_epochs_data(all_data,j)
        output_array = output_array + [r]
    return output_array

In [18]:
features = get_all_features(all_data)

In [19]:
def train_prep(features,targets,exclude=None,flatten_final=True):
    total_subjects = len(targets)
    target_list = []
    for i in range(total_subjects):
        num_epochs = features[i].shape[0]
        target_list.append(targets[i]*np.ones(num_epochs))
    if exclude==None: 
        features_array = np.concatenate(features)
        targets_array = np.concatenate(target_list)
    else:
        features_array = np.concatenate(features[:exclude] + features[exclude+1:])
        targets_array  = np.concatenate(target_list[:exclude] + target_list[exclude+1:])
    if flatten_final:
        features_array = features_array.reshape((features_array.shape[0],-1))
    return features_array, targets_array

In [20]:
train_prep(features, targets,exclude=10, flatten_final=True)[0].shape

(842, 30)

In [21]:
def kNN_cross(rbps,targets,n_neighbors, PCA_components = 0):
   
    confusion_matrices_train = []
    confusion_matrices_test = []
    labels = np.unique(targets)
    for i in range(len(targets)):
        train_X, train_y = train_prep(rbps,targets,exclude=i,flatten_final=True)
        test_X = rbps[i].reshape(rbps[i].shape[0],-1)
        test_y = targets[i]*np.ones(rbps[i].shape[0])

        scaler = StandardScaler()
        train_X = scaler.fit_transform(train_X)
        test_X = scaler.transform(test_X)
        
        if PCA_components != 0:
            pca = PCA(n_components = PCA_components)
            train_X = pca.fit_transform(train_X, y = None)
            test_X = pca.transform(test_X)
        
        ThreeNN = KNeighborsClassifier(n_neighbors=n_neighbors)
        ThreeNN.fit(train_X, train_y)
        
        confusion_matrices_train += [confusion_matrix(train_y, ThreeNN.predict(train_X),labels=labels)]
        confusion_matrices_test += [confusion_matrix(test_y,ThreeNN.predict(test_X),labels=labels)]
    
    confusion_matrices_train = np.array(confusion_matrices_train)
    confusion_matrices_test = np.array(confusion_matrices_test)
    total_confusion_train = np.sum(confusion_matrices_train, axis= 0)
    total_confusion_test = np.sum(confusion_matrices_test, axis= 0)
    
    train_metrics_dict = {'acc':accuracy(total_confusion_train), 'sens':sensitivity(total_confusion_train), 
                            'spec':specificity(total_confusion_train), 'f1':f1(total_confusion_train)}
    test_metrics_dict = {'acc':accuracy(total_confusion_test), 'sens':sensitivity(total_confusion_test), 
                            'spec':specificity(total_confusion_test), 'f1':f1(total_confusion_test)}
    
    
    return train_metrics_dict, test_metrics_dict


In [23]:
kNN_cross(features, targets, n_neighbors=3, PCA_components = 0 )

({'acc': 0.9965491435223682,
  'sens': 0.9956909937888199,
  'spec': 0.9975384890798424,
  'f1': 0.9967744442717238},
 {'acc': 0.6437718277066357,
  'sens': 0.6086956521739131,
  'spec': 0.6842105263157895,
  'f1': 0.6466512702078522})

In [24]:
PCA_components = [5, 10, 20]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = kNN_cross(features, targets, n_neighbors=3, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [25]:
val_acc, train_acc

([0.6740395809080326, 0.6647264260768335, 0.6414435389988359],
 [0.9850324297355729, 0.9968193913188093, 0.9965491435223682])

In [26]:
train_metric_dict, val_metric_dict = svm_cross(features, targets, kernel= "linear", reg_parameter = 1.0, degree=2, PCA_components= 0)
train_metric_dict, val_metric_dict

({'acc': 0.9208797605188758,
  'sens': 0.9197204968944099,
  'spec': 0.9222162549230218,
  'f1': 0.9256495409259621},
 {'acc': 0.7124563445867288,
  'sens': 0.6891304347826087,
  'spec': 0.7393483709273183,
  'f1': 0.7196367763904653})

In [27]:
train_metric_dict, val_metric_dict = svm_cross(features, targets, kernel= "linear", reg_parameter = 10.0, degree=2, PCA_components= 0)
train_metric_dict, val_metric_dict

({'acc': 0.9345168800931315,
  'sens': 0.9315993788819876,
  'spec': 0.9378804153240243,
  'f1': 0.9384116059906933},
 {'acc': 0.7206053550640279,
  'sens': 0.7,
  'spec': 0.7443609022556391,
  'f1': 0.7285067873303168})

In [28]:
train_metric_dict, val_metric_dict = svm_cross(features, targets, kernel= "poly", reg_parameter = 1.0, degree=2, PCA_components= 0)
train_metric_dict, val_metric_dict

({'acc': 0.8397014801263928,
  'sens': 0.7102096273291926,
  'spec': 0.9889903329752954,
  'f1': 0.8259407236856956},
 {'acc': 0.5681024447031432,
  'sens': 0.4891304347826087,
  'spec': 0.6591478696741855,
  'f1': 0.5481120584652863})

In [29]:
train_metric_dict, val_metric_dict = log_reg_cross(features, targets)
train_metric_dict, val_metric_dict

({'acc': 0.8957051388657908,
  'sens': 0.8836568322981366,
  'spec': 0.9095954171142141,
  'f1': 0.9007379854777121},
 {'acc': 0.7252619324796274,
  'sens': 0.6869565217391305,
  'spec': 0.7694235588972431,
  'f1': 0.7281105990783411})

In [30]:
train_metric_dict, val_metric_dict = log_reg_cross(features, targets, PCA_components = 15)
train_metric_dict, val_metric_dict

({'acc': 0.8669757192749044,
  'sens': 0.8541925465838509,
  'spec': 0.8817132116004296,
  'f1': 0.8730533457654691},
 {'acc': 0.7555296856810244,
  'sens': 0.7195652173913043,
  'spec': 0.7969924812030075,
  'f1': 0.7591743119266054})